In [43]:
import os
import pandas as pd
import pickle


#exp_name = 'preprocessing_type'
#experiments = ["adam", "other", "rmsprop", "sgd"]
general_single_path = os.path.join('..', '07_single_layer_rnns', 'mac_logs', 'residual_lstm_one_layer')
exp_path = os.path.join('logs_mac', 'multi_layer_smyl_layernorm')
no_bidi = "gridsearch_results_no_bidirectional.pkl"
bidi = "gridsearch_results_bidirectional.pkl"

def load_gridsearch_results(logs_path, pickle_filename: str = 'gridsearch_results.pkl'):
    results_path = os.path.join(logs_path, pickle_filename)
    with open(results_path, 'rb') as f:
        results = pickle.load(f)
    return results

def add_param_to_results(results, param_name, param_value):
    for res in results:
        if param_value not in res:
            res[param_name] = param_value
    return results

def get_non_layernorm_data(layernorm_results: list, general_results: list):
    non_layernorm_results = []
    for lres in layernorm_results:
        for gres in general_results:
            if gres['model-layer_normalization'] == False and gres["model-dilation_base"] is None and gres["model-residual_block_size"] is None and gres["model-smyl_std"] == False and gres["model-smyl_residual"] == False:
                has_same_params = True
                for key in lres.keys():
                    if key not in ["model-layer_normalization", "metrics_per_fold"] and key in gres:
                        if lres[key] != gres[key]:
                            has_same_params = False
                            break
                if has_same_params:
                    non_layernorm_results.append(gres)
    return non_layernorm_results

def remove_layernorm_excessive_results(layernorm_results: list, general_results: list, excessive_keys: list = ["model-rnn_cell_params-units", "model-num_layers", "model-bidirectional"]):
    """There are results with 100 units and 8 layers which are not in general results - remove them"""
    to_be_removed = []
    for i, lres in enumerate(layernorm_results):
        if lres["model-rnn_cell_params-units"] == 20 and lres["model-num_layers"] == 4 and lres["model-bidirectional"] == True:
            print("FOUND")
        all_vals_in_results = False
        for gres in general_results:
            all_vals_in_cur_res = True
            for key in excessive_keys:
                if lres[key] != gres[key]:
                    all_vals_in_cur_res = False
                    break
            if all_vals_in_cur_res:
                all_vals_in_results = True
                break
        if not all_vals_in_results:
            to_be_removed.append(i)
    for i in reversed(to_be_removed):
        layernorm_results.pop(i)

single_layer_results = load_gridsearch_results(general_single_path)
lstms_50 = []
for lstm_50 in single_layer_results:
    assert lstm_50["model-rnn_cell_params-units"] == 50# and lstm_50["model-cell_type"] == "lstm"
    lstm_50 = [lstm_50]
    #lstm_50 = add_param_to_results(lstm_50, 'model-layer_normalization', False)
    lstm_50 = add_param_to_results(lstm_50, 'model-bidirectional', False)
    lstm_50 = add_param_to_results(lstm_50, 'model-num_layers', 1)
    if lstm_50[0]["model-rnn_cell_params-layer_normalization"] != True: # Do NOT use layer normalization
        lstms_50.append(lstm_50[0])

no_bidi_results = load_gridsearch_results(exp_path, no_bidi)
#general_results = add_param_to_results(general_results, 'model-layer_normalization', False)

bidi_results = load_gridsearch_results(exp_path, bidi)
#non_layernorm_results = get_non_layernorm_data(layernorm_results, general_results) 
#remove_layernorm_excessive_results(layernorm_results, non_layernorm_results, excessive_keys=["model-rnn_cell_params-units", "model-num_layers", "model-bidirectional"])

print(bidi_results[0].keys())
print(len(bidi_results))
print(len(no_bidi_results))

def print_results_units_layers_and_bi(results):
    for i, res in enumerate(results):
        print(i)
        for key in res.keys():
            if key not in ["model-layer_normalization", "metrics_per_fold", "model-dilation_base", "model-residual_block_size", "model-smyl_std", "model-smyl_residual"]:
                print(key, res[key])

print("BIDI")
print_results_units_layers_and_bi(bidi_results)
print("NO_BIDI")
print_results_units_layers_and_bi(no_bidi_results)
    
grid_search_results = bidi_results + no_bidi_results + lstms_50

def remove_units20(results):
    to_be_removed = []
    for i, res in enumerate(results):
        if res["model-rnn_cell_params-units"] == 20:
            to_be_removed.append(i)
    for i in reversed(to_be_removed):
        results.pop(i)
    return results

grid_search_results = remove_units20(grid_search_results)
print(len(grid_search_results))
    

dict_keys(['model-num_layers', 'model-rnn_cell_params-units', 'model-dilation_base', 'model-residual_block_size', 'model-smyl_residual', 'model-layer_normalization', 'model-bidirectional', 'metrics_per_fold'])
24
24
BIDI
0
model-num_layers 2
model-rnn_cell_params-units 50
model-bidirectional True
1
model-num_layers 2
model-rnn_cell_params-units 50
model-bidirectional True
2
model-num_layers 2
model-rnn_cell_params-units 50
model-bidirectional True
3
model-num_layers 2
model-rnn_cell_params-units 50
model-bidirectional True
4
model-num_layers 2
model-rnn_cell_params-units 50
model-bidirectional True
5
model-num_layers 2
model-rnn_cell_params-units 50
model-bidirectional True
6
model-num_layers 2
model-rnn_cell_params-units 50
model-bidirectional True
7
model-num_layers 2
model-rnn_cell_params-units 50
model-bidirectional True
8
model-num_layers 4
model-rnn_cell_params-units 50
model-bidirectional True
9
model-num_layers 4
model-rnn_cell_params-units 50
model-bidirectional True
10
model-

In [ ]:
logs_folders = [
    "general_multi_layer_100",
    "general_multi_layer_100_pending",
    "general_multi_layer_part1",
    "multi_layer_layernorm",
    "run_pending_experiments copy",
    "run_pending_experiments_small",
    "multi_layer_smyl_layernorm"
]

for f in logs_folders:
    path = os.path.join('logs_mac', f)
    results = load_gridsearch_results(path)
    print(len(results))

In [ ]:
print(len(grid_search_results))
for exp in grid_search_results:
    for key in exp.keys():
        if key != 'metrics_per_fold':
            print(f"{key}: {exp[key]}")
    #print(f"epochs: {exp["training-epochs"]}")
    folds = exp["metrics_per_fold"]
    for i, fold in enumerate(folds):
        print(f"\tFold {i+1}:")
        for metric, values in fold.items():
            scaled_val = values["scaled"]
            unscaled_val = values["unscaled"]
            print(f"\t\t{metric}: {scaled_val.metric:.6f} ({unscaled_val.metric:.6f})\n\t\t\tper_sample s (u):\n\t\t\t\t {scaled_val.metric_per_sample}\n\t\t\t\t({unscaled_val.metric_per_sample})")
    print()
    print("######################################")

49
model-num_layers: 2
model-rnn_cell_params-units: 50
model-dilation_base: None
model-residual_block_size: None
model-smyl_residual: False
model-layer_normalization: True
model-bidirectional: True
	Fold 1:
		mean_squared_error: 0.000013 (0.603226)
			per_sample s (u):
				 [8.594242e-06, 3.7139273e-06, 8.532074e-06, 3.693397e-06, 2.9037587e-05, 1.0571763e-05, 3.8619482e-06, 1.1815475e-05, 2.7491358e-06, 5.788249e-06, 2.6132377e-05, 2.9434423e-05, 3.2914572e-06, 4.2184956e-06, 1.2171691e-05, 1.2465185e-05, 6.4888286e-06, 1.3589673e-05, 3.5269438e-06, 6.5283725e-06, 3.4734687e-06, 5.479038e-06, 3.6561396e-06, 5.808638e-06, 4.3157183e-06, 3.735502e-05, 3.0327425e-05, 1.9938921e-05, 3.0794145e-05, 2.9268402e-05, 2.651078e-05, 3.7594768e-06, 1.0798993e-05, 3.4284208e-06, 3.0980157e-06, 3.2446653e-05, 3.0180618e-06, 3.560352e-06, 3.0523537e-05, 4.786651e-06, 3.7588536e-05, 4.2193547e-06, 2.829597e-05, 1.7522918e-05, 3.6138772e-06]
				([0.46473914, 0.24502857, 0.45292106, 0.24213435, 1.1950

1. 
   - For each experiment, get all samples. 
   - The samples are ordered for each experiment in the same way. 

In [49]:
from sklearn.preprocessing import FunctionTransformer
from types import FunctionType
import re

def get_exp_name(exp):
    name = ""
    for i, key in enumerate(exp.keys()):
        if key != 'metrics_per_fold':
            if i > 0:
                name += " "
            if isinstance(exp[key], str):
                name += exp[key]
            elif isinstance(exp[key], FunctionTransformer):
                return exp[key].func.__name__
            elif isinstance(exp[key], bool):
                name += key if exp[key] else "not "+ key
            elif isinstance(exp[key], FunctionType):
                name += exp[key].__qualname__
            else:
                key_parts = key.split("-")
                last_part = key_parts[-1]
                key_words = last_part.split("_")
                shortcut = "".join([word[0] for word in key_words]) if len(key_words) > 1 else key_words[0]
                name += shortcut + "=" + str(exp[key])

    #name = name.replace("dataset-", "").replace("_", " ").replace("-", " ")
    name = name.capitalize()
    return name

def filter_arch_level_params(exp):
    for key, value in exp.items():
        if (
            key == "model-dilation_base" and value != None
            or key == "model-residual_block_size" and value != None
            or key == "model-smyl_std" and value != False
            or key == "model-smyl_residual" and value != False
        ):
            return True

    return False

def has_bidi(exp):
    for key, value in exp.items():
        if key == "model-bidirectional" and value == True:
            return True
    return False

def has_dilation_base(exp):
    for key, value in exp.items():
        if key == "model-dilation_base" and value != None:
            return True
    return False

def has_res_block_size(exp):
    for key, value in exp.items():
        if key == "model-residual_block_size" and value != None:
            return True
    return False

def has_smyl_residual(exp):
    for key, value in exp.items():
        if key == "model-smyl_residual" and value == True:
            return True
    return False 

def has_n_layers(exp, n):
    for key, value in exp.items():
        if key == "model-num_layers" and value == n:
            return True
    return False

exp_samples = {}
for i, exp in enumerate(grid_search_results):
    if has_bidi(exp) or has_dilation_base(exp): # or has_res_block_size(exp): #or not has_smyl_residual(exp):
    #if has_bidi(exp) or has_dilation_base(exp) or has_smyl_residual(exp):
        continue
    exp_name = get_exp_name(exp)
    exp_samples[exp_name] = []
    folds = exp["metrics_per_fold"]
    for fold in folds:
        exp_samples[exp_name].extend(fold["root_mean_squared_error"]["unscaled"].metric_per_sample)
    #print(len(exp_samples[exp_name]))

for exp_name, samples in exp_samples.items():
    print(f"{exp_name}: {len(samples)}")
    print(f"mean: {sum(samples)/len(samples)}")
    print(f"std: {pd.Series(samples).std()}")
    


Nl=2 units=50 db=none rbs=none not model-smyl_residual model-layer_normalization not model-bidirectional: 450
mean: 0.7471397269434399
std: 0.5777422785758972
Nl=2 units=50 db=none rbs=none model-smyl_residual model-layer_normalization not model-bidirectional: 450
mean: 0.5752899430526628
std: 0.3526943325996399
Nl=2 units=50 db=none rbs=2 not model-smyl_residual model-layer_normalization not model-bidirectional: 450
mean: 0.6539568093419075
std: 0.4105868339538574
Nl=2 units=50 db=none rbs=2 model-smyl_residual model-layer_normalization not model-bidirectional: 450
mean: 0.6447365680668089
std: 0.31152859330177307
Nl=4 units=50 db=none rbs=none not model-smyl_residual model-layer_normalization not model-bidirectional: 450
mean: 0.9813970000214047
std: 0.33339110016822815
Nl=4 units=50 db=none rbs=none model-smyl_residual model-layer_normalization not model-bidirectional: 450
mean: 0.8085738893349965
std: 0.4252118170261383
Nl=4 units=50 db=none rbs=2 not model-smyl_residual model-laye

In [51]:
from scipy import stats
import statsmodels.stats.multitest
from critdd import Diagram
from itertools import combinations
import numpy as np
import pandas as pd

samples = list(exp_samples.values())
print(np.array(samples).shape)
friedman_result = stats.friedmanchisquare(*samples)
friedman_result

(14, 450)


FriedmanchisquareResult(statistic=1663.4943492063467, pvalue=0.0)

In [35]:
combs = combinations(samples, 2)

pvals = []
for comb in combs:
    pvals.append(stats.wilcoxon(comb[0], comb[1]).pvalue)

pvals = np.array(pvals)
pvals.sort()
print(pvals)

for i in range(len(pvals)):
    if pvals[i] > 0.05/(len(pvals)-i):
        print(f"{pvals[i]} is not st. significant")
        print(f"{0.05/(len(pvals)-i)} is the threshold")
        break
    else:
        print(f"{pvals[i]} is st. significant")

statsmodels.stats.multitest.multipletests(pvals, method="holm")

[1.85247623e-75 2.97544195e-75 5.45635847e-75 8.92708807e-75
 3.59554253e-74 1.92384838e-71 2.09332411e-71 1.32553361e-69
 1.35995456e-69 2.21004339e-68 3.96222856e-68 3.70703963e-67
 2.86423348e-66 5.68286981e-64 7.39873069e-64 8.40014774e-62
 4.20980884e-61 1.96908977e-60 3.34590530e-60 2.46038295e-58
 3.00219065e-58 1.07046579e-56 1.12823135e-55 1.75156082e-55
 8.71371249e-54 6.60066127e-53 1.86612696e-52 4.01408741e-51
 8.24667779e-50 1.87107370e-49 7.78897001e-48 1.48346930e-47
 2.32198433e-45 2.49541158e-45 1.58609562e-40 1.46451771e-38
 7.41418834e-37 1.91082670e-35 1.70100484e-34 2.61021181e-34
 1.79190367e-32 2.36327948e-32 1.58186823e-31 1.95731958e-31
 3.15636606e-28 3.26171334e-26 3.39041979e-26 1.87272580e-25
 2.21439147e-25 3.36299596e-25 2.77469274e-24 4.15817586e-24
 2.57035957e-22 1.29852890e-20 1.53752698e-19 1.94280453e-18
 1.58551343e-17 5.81006606e-16 6.22140069e-16 6.52433796e-16
 1.26235879e-14 1.60607716e-14 1.10397078e-12 1.65533469e-11
 2.88188381e-10 1.546413

(array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False, False, False, False, False, False, False, False,
        False]),
 array([1.68575337e-73, 2.67789775e-73, 4.85615904e-73, 7.85583750e-73,
        3.12812200e-72, 1.65450960e-69, 1.77932549e-69, 1.11344824e-67,
        1.12876228e-67, 1.81223558e-66, 3.20940513e-66, 2.96563171e-65,
        2.26274445e-64, 4.43263845e-62, 5.69702263e-62, 6

In [19]:
diag_df = pd.DataFrame(exp_samples)
diag_df.columns

Index(['Nl=2 units=50 db=none rbs=none not model-smyl_residual model-layer_normalization not model-bidirectional',
       'Nl=2 units=50 db=none rbs=none model-smyl_residual model-layer_normalization not model-bidirectional',
       'Nl=4 units=50 db=none rbs=none not model-smyl_residual model-layer_normalization not model-bidirectional',
       'Nl=4 units=50 db=none rbs=none model-smyl_residual model-layer_normalization not model-bidirectional',
       'Nl=6 units=50 db=none rbs=none not model-smyl_residual model-layer_normalization not model-bidirectional',
       'Nl=6 units=50 db=none rbs=none model-smyl_residual model-layer_normalization not model-bidirectional',
       'Lstm units=50 not model-layer_normalization not model-bidirectional nl=1'],
      dtype='object')

In [14]:
exp_samples.values()

dict_values([[0.68172026, 0.49500364, 0.6729941, 0.4920723, 1.0931613, 0.7408419, 0.50774384, 0.76349884, 0.32295898, 0.531684, 1.1276609, 1.078276, 0.38688308, 0.404884, 0.77453834, 0.7876756, 0.5352587, 0.8409555, 0.47253242, 0.5402599, 0.39209023, 0.5378772, 0.4865675, 0.5279496, 0.40668428, 1.4236023, 1.0589013, 1.1099813, 1.0381569, 1.0838536, 1.1287224, 0.49824893, 0.7429542, 0.43810314, 0.33673766, 0.9876011, 0.33214477, 0.45369324, 0.9818189, 0.5470131, 1.3521619, 0.53061986, 1.0960934, 1.0274245, 0.48377487, 0.47506088, 0.43615523, 1.1254421, 0.48320892, 0.44818717, 0.76278704, 0.4646353, 1.2475972, 0.4587176, 0.4789717, 1.0782881, 0.41154626, 0.47831652, 1.1149324, 1.1056645, 0.47865564, 0.45674634, 0.44508922, 1.0894725, 1.1054361, 0.9979597, 1.1691581, 1.1896622, 0.40311563, 0.44755054, 1.1274124, 1.1743299, 1.3356599, 0.98160076, 1.33871, 1.4168497, 0.41927248, 0.4475959, 1.1482931, 0.52944005, 1.1247953, 0.4502403, 1.1122569, 0.90564436, 0.39333668, 1.1225367, 0.46530104,

In [50]:
x=np.array(list(exp_samples.values()))
treatment_names = list(exp_samples.keys())
print(treatment_names)
x = x.T
diagram = Diagram(x,
    treatment_names=treatment_names,
    maximize_outcome = False,
)

diagram.to_file(
    "critdd_layernorm_multilayer_residual_both.tex",
    alpha=0.05,
    adjustment="holm",
    reverse_x=True,
)

['Nl=2 units=50 db=none rbs=none not model-smyl_residual model-layer_normalization not model-bidirectional', 'Nl=2 units=50 db=none rbs=none model-smyl_residual model-layer_normalization not model-bidirectional', 'Nl=2 units=50 db=none rbs=2 not model-smyl_residual model-layer_normalization not model-bidirectional', 'Nl=2 units=50 db=none rbs=2 model-smyl_residual model-layer_normalization not model-bidirectional', 'Nl=4 units=50 db=none rbs=none not model-smyl_residual model-layer_normalization not model-bidirectional', 'Nl=4 units=50 db=none rbs=none model-smyl_residual model-layer_normalization not model-bidirectional', 'Nl=4 units=50 db=none rbs=2 not model-smyl_residual model-layer_normalization not model-bidirectional', 'Nl=4 units=50 db=none rbs=2 model-smyl_residual model-layer_normalization not model-bidirectional', 'Nl=6 units=50 db=none rbs=none not model-smyl_residual model-layer_normalization not model-bidirectional', 'Nl=6 units=50 db=none rbs=none model-smyl_residual mod

In [29]:
def get_models_without_dropout(diagram: Diagram):
    """Gets the best optimizer for each type (Adam, RMSprop, SGD, ...)"""
    sorted_treatments = sorted(zip(diagram.treatment_names, diagram.average_ranks), key=lambda x: x[1])
    no_dropout_models = {}
    for name, avg_rank in sorted_treatments:
        if name.endswith("dropout=0.0"):
            #name = re.sub(" dropout=0.0", "", name)
            no_dropout_models[name] = (name, avg_rank)
    return no_dropout_models

def get_partial_diagram(subset_names, samples):
    x = np.array([samples[name] for name in subset_names])
    x = x.T
    return Diagram(x,
        treatment_names=subset_names,
        maximize_outcome = False,
    )

no_dropout_models = get_models_without_dropout(diagram)
partial_diagram = get_partial_diagram([name for name, _ in no_dropout_models.values()], exp_samples)

print("Previous average ranks:")
for name, avg_r in no_dropout_models.values():
    print(f"{name}: {avg_r}")

print("\nNew average ranks:")
sorted_treatments = sorted(zip(partial_diagram.treatment_names, partial_diagram.average_ranks), key=lambda x: x[1])
for name, avg_r in sorted_treatments:
    print(f"{name}: {avg_r}")

partial_diagram.to_file(
    "critdd_layernorm_no_drop.tex",
    alpha=0.05,
    adjustment="holm",
    reverse_x=True,
)

Previous average ranks:
Not model-layer_normalization rd=0.0 dropout=0.0: 4.302222222222222
Model-layer_normalization rd=0.2 dropout=0.0: 4.377777777777778
Model-layer_normalization rd=0.1 dropout=0.0: 4.988888888888889
Model-layer_normalization rd=0.0 dropout=0.0: 5.015555555555555
Not model-layer_normalization rd=0.2 dropout=0.0: 5.3933333333333335
Model-layer_normalization rd=0.05 dropout=0.0: 5.595555555555555
Model-layer_normalization rd=0.3 dropout=0.0: 6.0311111111111115
Not model-layer_normalization rd=0.3 dropout=0.0: 6.153333333333333
Not model-layer_normalization rd=0.1 dropout=0.0: 6.388888888888889
Not model-layer_normalization rd=0.05 dropout=0.0: 6.753333333333333

New average ranks:
Not model-layer_normalization rd=0.0 dropout=0.0: 4.302222222222222
Model-layer_normalization rd=0.2 dropout=0.0: 4.377777777777778
Model-layer_normalization rd=0.1 dropout=0.0: 4.988888888888889
Model-layer_normalization rd=0.0 dropout=0.0: 5.015555555555555
Not model-layer_normalization rd